# Python State Machine
## Eine selbstgeschriebene SM

In [13]:
class StateMachine:
    def __init__(self):
        self.state = None  # Der aktuelle Zustand des Automaten
        self.transitions = {}  # Eine Tabelle der Übergänge zwischen Zuständen

    def add_state(self, state_name, is_initial=False):
        """Fügt einen neuen Zustand hinzu. Falls is_initial True ist, wird dieser Zustand als Anfangszustand festgelegt."""
        self.transitions[state_name] = {}
        if is_initial:
            self.state = state_name

    def add_transition(self, from_state, to_state, event):
        """Definiert einen Übergang von einem Zustand zu einem anderen bei einem bestimmten Ereignis."""
        if from_state in self.transitions:
            self.transitions[from_state][event] = to_state
        else:
            raise ValueError(f"Zustand '{from_state}' nicht definiert")

    def on_event(self, event):
        """Reagiert auf ein Ereignis und verändert den Zustand des Automaten entsprechend."""
        if self.state in self.transitions and event in self.transitions[self.state]:
            self.state = self.transitions[self.state][event]
        else:
            raise ValueError(f"Kein Übergang definiert für Zustand '{self.state}' bei Ereignis '{event}'")

    def get_state(self):
        """Gibt den aktuellen Zustand des Automaten zurück."""
        return self.state

# Beispiel für die Nutzung des Zustandsautomaten
fsm = StateMachine()
fsm.add_state("wait", is_initial=True)
fsm.add_state("wait_for_value")
fsm.add_state("wait_for_triggerId")
fsm.add_state("write_to_db")

fsm.add_transition("wait", "wait_for_value", "new_triggerId_event")
fsm.add_transition("wait", "wait_for_triggerId", "new_value_event")
fsm.add_transition("wait_for_triggerId", "write_to_db" , "new_triggerId_event")
fsm.add_transition("wait_for_value", "write_to_db" , "new_value_event")
fsm.add_transition("write_to_db" ,"wait", "db_ok")



print(fsm.get_state())  # Ausgabe: initial
fsm.on_event("new_triggerId_event")
print(fsm.get_state())  # Ausgabe: state1
fsm.on_event("new_value_event")
print(fsm.get_state())  # Ausgabe: state2
fsm.on_event("db_ok")
print(fsm.get_state())  # Ausgabe: state1

wait
wait_for_value
write_to_db
wait


# Eine StateMachine (SM) mit Transitions Library
https://github.com/pytransitions/transitions?tab=readme-ov-file#table-of-contents

In [1]:
import transitions import Machine

class Watch_PV_SM:
    '''
    Watch_PV_SM Klasse
    ==================

    Das ist der Observer für Laser ProzessVariablen und TriggerID.
    Die TriggerID wird von Raspberry PI erzeugt, nachdem es Trigger Signale am GPIO registriert. Dadurch entsteht 
    dem Triggersignal und Publizieren der TriggerId ein Delay.
    Weil es möglich ist, dass schnelle Geräte ihre Werte kurz vor der TriggerID publizieren und langsame Geräte 
    dagegen danach, wird Watch_PV_SM als StateMachine realisiert:

    States
    -------
    - Grundzustand: watch_PV
    - waiting_for_PV_update
    - waiting_for_trigger
    - write_to_db
    - timeout
    - error

    Szenarien
    ----------

    1. Szenario 1
        - Wenn neue TriggerID registriert wird, erfolgt ein Wechsel in Zustand: waiting_for_PV_update
        - waiting_for_PV_update wartet auf Änderung der PV. Passiert diese Änderung werden die Daten in DB geschrieben
        - write_to_db und Übergang in Grundzustand

    2. Szenario 2    
        - Wenn zuerst ein PV Update erfolgt, wechselt der Automat in Zustand: waiting_for_trigger
        - u.s.w  
    '''


    states=['watch_PV', 'waiting_for_trigger','waiting_for_PV_update', 'write_to_db', 'timeout', 'error']

    def __init__(self) -> None:
        self.SM = Machine(model=Watch_PV_SM, states=Watch_PV_SM.states)

In [8]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import logging


class SimpleMachine(object):

    states = ['dummy', 'start', 'waiting']

    def __init__(self, stop_event):
        self.machine = AsyncMachine(model=self, states=SimpleMachine.states, initial='dummy')
        self.machine.add_transition('run', 'dummy', 'start')
        self.machine.add_transition('timeoutTransition', '*', 'waiting')
        self.machine.add_transition('stop_waiting', 'waiting', 'start')
        self.stop_event = stop_event


    async def doing_things(self):
        try:
            await asyncio.wait_for(self.stop_event.wait(), timeout=10.0)
            print("Event was set!")
        except asyncio.TimeoutError:
            print("Timeout reached, event was not set .")
        
        # print("work work")
        # await asyncio.sleep(10)

    async def on_enter_start(self):
        try:
            await asyncio.wait_for(self.doing_things(), 5)
        except asyncio.TimeoutError:
            print("Timeout!")
            await self.timeoutTransition()

    async def on_enter_waiting(self):
        print("Enter wait state")
        
test_machine = None
test_SM = None

# def main():
#     # logging.basicConfig(level=logging.DEBUG)
#     global test_machine
#     global test_SM
#     test_machine = SimpleMachine()
#     print("State Machine started")
#     asyncio.get_event_loop().run_until_complete(test_machine.run())

async def main():
    global test_machine
    global test_SM
    test_machine = SimpleMachine()
    print("State Machine started")
    test_SM = asyncio.create_task(test_machine.run())

asyncio.run(main())

State Machine started


work work
Timeout!
Enter wait state


In [6]:
test_machine.state

'waiting'

In [7]:
test_SM.cancel()

False

In [37]:
class Dancer:
    
    states = ['start', 'left_food_left', 'left', 'right_food_right']
    
    def __init__(self, name, beat):
        self.my_name = name
        self.my_beat = beat
        self.moves_done = 0
        
    async def on_enter_start(self):
        self.moves_done += 1
        
    async def wait(self):
        print(f'{self.my_name} stepped {self.state}')
        await asyncio.sleep(self.my_beat)

    async def dance(self):
        while self.moves_done < 2:
            await self.step()
        
dancer1 = Dancer('Tick', 1)
dancer2 = Dancer('Tock', 2)

m = AsyncMachine(model=[dancer1, dancer2], states=Dancer.states, initial='start', after_state_change='wait')
m.add_ordered_transitions(trigger='step')

# it starts okay but becomes quite a mess
await asyncio.gather(dancer1.dance(), dancer2.dance()) 
#await asyncio.gather(dancer1.dance())

Tick stepped left_food_left
Tock stepped left_food_left
Tick stepped left
Tock stepped left
Tick stepped right_food_right
Tick stepped start
Tock stepped right_food_right
Tick stepped left_food_left
Tick stepped left
Tick stepped right_food_right
Tock stepped start
Tick stepped start
Tock stepped left_food_left
Tock stepped left
Tock stepped right_food_right
Tock stepped start


[None, None]